# 0. Import library

In [ ]:
# import library
import pandas as pd
import numpy as np
import os
import openpyxl
import re

import matplotlib
import matplotlib.pyplot
import seaborn as sns
import matplotlib.pyplot as plt
# 나눔 고딕 적용
matplotlib.rc("font", family = "NanumGothic")
# 음수 표시
matplotlib.rc("axes", unicode_minus = False)

# 데이터 분할:train, test
from sklearn.model_selection import train_test_split

# 1. Import data

In [ ]:
cfg = {'file_path' : '/home/piai/AIweek/AI_PROJECT/Develop/DialectNLP/',
       'data_path' : '/home/piai/AIweek/AI_PROJECT/Develop/DialectNLP/dataset/'}

# 2. Preprocessing!!

In [ ]:
# # 텍스트 파일 열기
# # DKSR20000890 ~ DKSR20001736
# for _ in range
#     with open(cfg['file_path']+'(new3)라벨링데이터/DKSR20000890.txt', 'r', encoding='utf-8') as file:
#         # 파일의 모든 내용을 읽고 출력하기
#         content = file.read()
#         # print(content)

stand_list = list()
gsd_list = list()
# DKSR20000890부터 DKSR20001736까지의 파일을 순회
for i in range(890, 1737):
    # 파일 이름 생성
    file_name = f'DKSR2000{i:04}.txt'
    
    # 파일 전체 경로
    full_path = os.path.join(cfg['file_path'], '(new3)라벨링데이터/', file_name)

    # 파일 열기 및 내용 읽기
    try:
        with open(full_path, 'r', encoding='utf-8') as file:
            content = file.read()
            # 파일 내용 처리 (예: 출력)
            # print(content)
    except FileNotFoundError:
        print(f"파일이 존재하지 않습니다: {file_name}")

    # (())라는 문자열을 제거
    p = re.compile('\(\(\)\) ')
    content_2 = p.sub('', content)
    # print(content_2)
    
    # 줄바꿈(\n)을 띄어쓰기(' ')로 바꾸기
    p = re.compile('\\n')
    content_3 = p.sub(' ', content_2)
    # print(content_3)
    
    def replace_with_newline(match, replacements=[0]):
        if replacements[0] == 0:
            # 첫 번째 매치는 바꾸지 않음
            replacements[0] += 1
            return match.group()[2:]
        else:
            # 두 번째부터 '1:'를 지우고 줄바꿈으로 대체
            return '\n' + match.group()[2:]  # '1:' 부분을 제외하고 나머지를 반환
    
    p = re.compile('1: ')
    # 커스텀 교체 함수 사용
    content_4 = p.sub(replace_with_newline, content_3)
    # print(content_4)
    
    p = re.compile('2: ')
    # 커스텀 교체 함수 사용
    content_5 = p.sub(replace_with_newline, content_4)
    # print(content_5)
    
    # '--' 사이의 한글 단어를 찾아 제거하는 정규 표현식
    pattern = r'-[가-힣]+- '
    # 정규 표현식을 사용하여 문장에서 패턴에 해당하는 부분 제거
    content_6 = re.sub(pattern, '', content_5)
    # print(content_6)
    
    # 한 글자 + '~' 패턴을 찾아 제거하는 정규 표현식
    pattern = r'[가-힣]~ '
    # 정규 표현식을 사용하여 패턴에 해당하는 부분 제거
    content_7 = re.sub(pattern, '', content_6)
    # print(content_7)
    
    # 한 글자 + '~' 패턴을 찾아 제거하는 정규 표현식
    pattern = r'@\S+ '
    # 정규 표현식을 사용하여 패턴에 해당하는 부분 제거
    content_8 = re.sub(pattern, '', content_7)
    # print(content_8)
    
    # 한 글자 + '~' 패턴을 찾아 제거하는 정규 표현식
    pattern = '\n'
    # 정규 표현식을 사용하여 패턴에 해당하는 부분 제거
    content_9 = re.sub(pattern, '', content_8)
    # print(content_9)
    

    # '(A)/(B)' 형식에서 A만 남기는 정규 표현식
    pattern_A = r'\(([^)]+)\)/\([^)]+\)'
    # '(A)/(B)' 형식에서 B만 남기는 정규 표현식
    pattern_B = r'\([^)]+\)/\(([^)]+)\)'
    
    # 정규 표현식을 사용하여 첫 번째 괄호 안의 내용만 남김
    stand = re.sub(pattern_B, r'\1', content_9)
    gsd = re.sub(pattern_A, r'\1', content_9)
    # print(stand)
    
    # 한 글자 + '~' 패턴을 찾아 제거하는 정규 표현식
    pattern = '\n'
    # 정규 표현식을 사용하여 패턴에 해당하는 부분 제거
    stand = re.sub(pattern, '', stand)
    gsd = re.sub(pattern, '', gsd)

    stand_list.append(stand)
    gsd_list.append(gsd)

    # 빈 DataFrame 생성
    globals()[f'df_{i}'] = pd.DataFrame(columns=['stand', 'gsd'])
    
    # 문장 분리 및 구두점 유지
    sentences = re.findall(r'[^.!?]+[.!?]', stand)
    sentences = [sentence for sentence in sentences if sentence]
    # 1열에 문장 저장
    globals()[f'df_{i}']['stand'] = pd.Series(sentences)
    
    # 문장 분리 및 구두점 유지
    sentences = re.findall(r'[^.!?]+[.!?]', gsd)
    sentences = [sentence for sentence in sentences if sentence]
    # 1열에 문장 저장
    globals()[f'df_{i}']['gsd'] = pd.Series(sentences)

# 연결할 데이터프레임 목록 생성 (제외할 데이터프레임은 건너뜀)
dataframes_to_concat = [globals()[f'df_{i}'] for i in range(890, 1737) 
                        if i not in (1271, 1273, 1294, 1702)]

# 데이터프레임 연결
df_combined = pd.concat(dataframes_to_concat, ignore_index=True)

df_combined.shape

In [ ]:
df_combined.isnull().sum(axis = 0)

In [ ]:
df_combined.isnull().sum(axis = 0)

In [ ]:
df_combined

In [ ]:
# 데이터프레임을 넘파이 배열로 변환
numpy_array = df_combined.to_numpy()
numpy_array

In [ ]:
# # 넘파이 행렬을 리스트로 변환
# df2 = numpy_array.tolist()
# df2
df2 = numpy_array

In [ ]:
print(len(df2))
# 중복된 리스트를 제거하여 유니크한 리스트만 남김
df3 = [list(x) for x in set(tuple(x) for x in df2)]

print(len(df3))

In [ ]:
df3

In [ ]:
def contains_float(lst):
    for item in lst:
        if isinstance(item, float):
            return True
    return False

# 리스트의 값 중에서 float 값이 있는지 확인
contains_float_value = contains_float(df3)

if contains_float_value:
    print("리스트의 값 중에 float 값이 포함되어 있습니다.")
else:
    print("리스트의 값 중에 float 값이 포함되어 있지 않습니다.")

In [ ]:
# 데이터 형식 변경
df4 = [{'standard': pair[0], 'dialect': pair[1]} for pair in df3]
df4

In [ ]:
# 데이터셋에서 문자열이 아닌 부분을 찾아서 출력
for d in df4:
    if not isinstance(d['standard'], str) or not isinstance(d['dialect'], str):
        print("문자열이 아닌 데이터 발견:")
        print("standard:", d['standard'])
        print("dialect:", d['dialect'])

In [ ]:
# 문자열이 아닌 데이터를 제외한 데이터셋을 생성
df5 = []
print(len(df4))
for d in df4:
    if isinstance(d['standard'], str) and isinstance(d['dialect'], str):
        df5.append(d)
print(len(df5))
type(df5)

In [ ]:
temp = pd.DataFrame(df5)
# nan_locations = temp.isna()
# print(nan_locations)
nan_count = temp.isna().sum().sum()
print(f"NaN 값의 개수: {nan_count}")

In [ ]:
# 데이터를 8:2 비율로 분할
train, test = train_test_split(df5, test_size=0.3, random_state=42)

# 분할된 데이터의 크기 확인
print(len(train), len(test))

In [ ]:
import json

# JSON 파일로 저장
with open(cfg['data_path']+'sent_gs_train.json', 'w', encoding='utf-8') as f:
    json.dump(train[:1000], f, ensure_ascii=False, indent=4)
    
with open(cfg['data_path']+'sent_gs_test.json', 'w', encoding='utf-8') as f:
    json.dump(test[:100], f, ensure_ascii=False, indent=4)

print("딕셔너리가 'sent_gs_train.json' 파일로 저장되었습니다.")

In [ ]:
# def readJson(self, path):
#     with open(path, 'r', encoding='utf8') as f:
#         data = json.load(f)
#     # 문자열이 아닌 데이터 필터링
#     data = [d for d in data if isinstance(d['standard'], str) and isinstance(d['dialect'], str)]
#     self.pairs = [[self._normalize(d['standard']), self._normalize(d['dialect'])] for d in data]
#     self._filterPairs()
#     self.srcs = [pair[0] for pair in self.pairs]
#     self.trgs = [pair[1] for pair in self.pairs]

# def readJson(self, path):
#     with open(path, 'r', encoding='utf8') as f:
#         data = json.load(f)
#     self.pairs = [[self._normalize(d['standard']), self._normalize(d['dialect'])] for d in data if isinstance(d['standard'], str) and isinstance(d['dialect'], str)]
#     self._filterPairs()
#     self.srcs = [pair[0] for pair in self.pairs]
#     self.trgs = [pair[1] for pair in self.pairs]

# def readJson(self, path):
#     with open(path, 'r', encoding='utf8') as f:
#         data = json.load(f)
#     self.pairs = []
#     for d in data:
#         standard = d.get('standard', None)
#         dialect = d.get('dialect', None)
#         if standard is not None and dialect is not None and isinstance(standard, str) and isinstance(dialect, str):
#             self.pairs.append([self._normalize(standard), self._normalize(dialect)])
#     self._filterPairs()
#     self.srcs = [pair[0] for pair in self.pairs]
#     self.trgs = [pair[1] for pair in self.pairs]

# 데이터프레임을 사용하여 JSON 파일 읽기
def readJson(self, path):
    df = pd.read_json(path, encoding='utf8')
    df = df.fillna('')  # NaN 값을 빈 문자열로 대체
    self.pairs = [[self._normalize(d['standard']), self._normalize(d['dialect'])] for _, d in df.iterrows()]  # 수정
    self._filterPairs()
    self.srcs = [pair[0] for pair in self.pairs]
    self.trgs = [pair[1] for pair in self.pairs]

In [ ]:
# JSON 파일 경로
train_path = cfg['data_path']+'sent_gs_train.json'
test_path = cfg['data_path']+'sent_gs_test.json'

# Loader 인스턴스 생성
loader = Loader(MAX_LENGTH, INPUT_LEVEL)

# JSON 파일 읽기
loader.readJson(train_path)

# 데이터 확인
print(loader.pairs)  # [ [standard_text, dialect_text], ... ]
print(loader.srcs)   # [ standard_text, ... ]
print(loader.trgs)   # [ dialect_text, ... ]

In [ ]:
# 데이터 형식 확인
for d in df4[:10]:
    print(type(d['standard']), type(d['dialect']))

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
from torch.nn import functional as F

# 간단한 Encoder 클래스 정의
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src sent len, batch size]
        
        embedded = self.dropout(self.embedding(src))        
        #embedded = [src sent len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)                
        #outputs = [src sent len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src sent len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

# 간단한 Attention 클래스 정의
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Parameter(torch.rand(dec_hid_dim))
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat encoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src sent len, dec hid dim]
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2)))         
        #energy = [batch size, src sent len, dec hid dim]
        
        #energy = energy.squeeze(1)  # 새로운 차원 제거
        energy = energy.permute(0, 2, 1)        
        #energy = [batch size, dec hid dim, src sent len]
        
        #v = [dec hid dim]
        
        v = self.v.repeat(batch_size, 1).unsqueeze(1)        
        #v = [batch size, 1, dec hid dim]
                
        attention = torch.bmm(v, energy).squeeze(1)
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

# 간단한 Decoder 클래스 정의
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention        
        self.embedding = nn.Embedding(output_dim, emb_dim)        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)        
        self.out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)        
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)        
        #weighted = [1, batch size, enc hid dim * 2]
        
        #weighted = weighted.squeeze(1)  # 새로운 차원 제거
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))        
        #output = [sent len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #sent len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        output = self.out(torch.cat((output, weighted, embedded), dim = 1))        
        #output = [bsz, output dim]
        
        return output, hidden.squeeze(0)

# Seq2Seq 모델 정의
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, sos_idx, device, max_length):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.sos_idx = sos_idx
        self.device = device
        self.max_length = max_length
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src sent len, batch size]
        #trg = [trg sent len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        if trg is None:
            assert teacher_forcing_ratio == 0, "Must be zero during inference"
            inference = True
            trg = torch.zeros((self.max_length, src.shape[1])).long().fill_(self.sos_idx).to(src.device)
        else:
            inference = False

        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)

        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, max_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs
    
class Vocab:
    def __init__(self, sentences, input_level, device):
        self.sentences = sentences
        self.stoi = {'<pad>': 0, '<sos>': 1, '<eos>':2, '<unk>':3}
        self.s_freq = {'<pad>': 0, '<sos>': 0, '<eos>':0, '<unk>':0}
        self.itos = {0: '<pad>', 1: '<sos>', 2: '<eos>', 3: '<unk>'}
        self.vocab_size = 4
        self.PAD_IDX = 0
        self.SOS_IDX = 1
        self.EOS_IDX = 2
        self.UNK_IDX = 3
        self.input_level = input_level
        self.device = device
    
    def _addWord(self, s):
        if s not in self.stoi:
            self.stoi[s] = self.vocab_size
            self.s_freq[s] = 1
            self.itos[self.vocab_size] = s
            self.vocab_size += 1
        else:
            self.s_freq[s] += 1

    def build_vocab(self):
        for sentence in self.sentences:
            if self.input_level == 'syl':
                sentence = [ch for ch in sentence]
            elif self.input_level == 'word':
                sentence = sentence.split()
            elif self.input_level == 'jaso':
                print("NOT IMPLEMENTED!")
                exit()

            for s in sentence:
                self._addWord(s)
    
    def sentenceFromIndex(self, indexs):
        ret_list = []
        for t in indexs:
            if t in self.itos:
                ret_list.append(self.itos[t])
            else:   
                ret_list.append('<unk>')
        return ret_list

    def indexesFromSentence(self, sentence, sos, eos):
        ret_list = []
        if sos:
            ret_list.append(self.SOS_IDX)

        if self.input_level == 'syl':
            sentence = [ch for ch in sentence]
        elif self.input_level == 'word':
            sentence = sentence.split()
        elif self.input_level == 'jaso':
            print("NOT IMPLEMENTED!")
            exit()

        for s in sentence:
            if s in self.stoi:
                ret_list.append(self.stoi[s])
            else:
                ret_list.append(self.UNK_IDX)
        if eos:
            ret_list.append(self.EOS_IDX)
        return ret_list

    def tensorFromSentence(self, sentence, sos, eos):
        indexes = self.indexesFromSentence(sentence, sos, eos)
        return torch.tensor(indexes, dtype=torch.long, device=self.device).view(-1, 1)
        # return = [sent len, batch size]
        # return = [sent len, 1]
        
class Loader:
    def __init__(self, max_len, input_level):
        self.pairs = []
        self.srcs = []
        self.trgs = []
        self.max_len = max_len
        self.input_level = input_level

    def _normalize(self, sentence):
        sentence = sentence.strip()
        sentence = re.sub(r"[^가-힣 ]", r"", sentence)
        return sentence
    
    def _filterPairs(self):
        filtered = []
        for pair in self.pairs:
            if self.input_level == 'syl':
                len_p0 = len(pair[0])
                len_p1 = len(pair[1])
            if self.input_level == 'word':
                len_p0 = len(pair[0].split())
                len_p1 = len(pair[1].split())

            if len_p0 < self.max_len and len_p1 < self.max_len:
                filtered.append(pair)
        self.pairs = filtered

    def readJson(self, path):
        with open(path, 'r', encoding='utf8') as f:
            data = json.load(f)
        self.pairs = [[self._normalize(d['standard']), self._normalize(d['dialect'])] for d in data]
        self._filterPairs()
        self.srcs = [pair[0] for pair in self.pairs]
        self.trgs = [pair[1] for pair in self.pairs]
        
    def makeIterator(self, SRC, TRG, batch_size):
        src_tensors = []
        trg_tensors = []
        sos = 1  # 시작 토큰
        eos = 2  # 종료 토큰

        for pair in self.pairs:
            src = SRC.tensorFromSentence(pair[0], sos, eos)
            trg = TRG.tensorFromSentence(pair[1], sos, eos)
            src_tensors.append(src)
            trg_tensors.append(trg)

        dataset = TensorDataset(src_tensors, trg_tensors)
        iterator = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return iterator
        
#     def makeIterator(self, SRC, TRG, batch_size):
#         src_tensors = []
#         trg_tensors = []
#         for pair in self.pairs:
#             src = SRC.tensorFromSentence(pair[0], sos, eos)
#             trg = TRG.tensorFromSentence(pair[1], sos, eos)
#             src_tensors.append(src)
#             trg_tensors.append(trg)

#         # 패딩을 적용하여 텐서 길이를 맞춰줍니다.
#         src_tensors = pad_sequence(src_tensors, batch_first=True)
#         trg_tensors = pad_sequence(trg_tensors, batch_first=True)

#         dataset = TensorDataset(src_tensors, trg_tensors)
#         iterator = DataLoader(dataset, batch_size=batch_size, shuffle=True)
#         return iterator
        
#     def makeIterator(self, SRC, TRG, batch_size):
#         src_tensors = []
#         trg_tensors = []
#         sos = True  # 기본값 설정
#         eos = True  # 기본값 설정
#         for pair in self.pairs:
#             src = SRC.tensorFromSentence(pair[0], sos, eos)
#             trg = TRG.tensorFromSentence(pair[1], sos, eos)
#             src_tensors.append(src)
#             trg_tensors.append(trg)

#         dataset = TensorDataset(src_tensors, trg_tensors)
#         iterator = DataLoader(dataset, batch_size=batch_size, shuffle=True)
#         return iterator
        
#     def makeIterator(self, batch_size, sos=True, eos=True, batch_size):

#         # 입력 문장과 출력 문장을 각각 정리합니다.
#         src_tensors = [self.src_vocab.tensorFromSentence(pair[0], sos, eos) for pair in self.pairs]
#         trg_tensors = [self.trg_vocab.tensorFromSentence(pair[1], sos, eos) for pair in self.pairs]

#         # 패딩을 추가합니다.
#         padded_src_tensors = pad_sequence(src_tensors, padding_value=self.src_vocab.PAD_TOKEN, batch_first=True)
#         padded_trg_tensors = pad_sequence(trg_tensors, padding_value=self.trg_vocab.PAD_TOKEN, batch_first=True)

#         # 배치를 생성합니다.
#         dataset = TensorDataset(padded_src_tensors, padded_trg_tensors)
#         iterator = DataLoader(dataset, batch_size=batch_size, shuffle=True)

#         return iterator
        
#     def makeIterator(self, SRC, TRG, sos, eos, batch_size):
#         src_tensors = [SRC.tensorFromSentence(pair[0], sos, eos) for pair in self.pairs]
#         trg_tensors = [TRG.tensorFromSentence(pair[1], sos, eos) for pair in self.pairs]

#         # 입력 데이터의 차원을 조정합니다.
#         src_tensors = torch.stack(src_tensors)
#         trg_tensors = torch.stack(trg_tensors)

#         dataset = TensorDataset(src_tensors, trg_tensors)
#         iterator = DataLoader(dataset, batch_size=batch_size, shuffle=True)
#         return iterator
    

    


MAX_LENGTH = 150
INPUT_LEVEL = 'syl'
PATH_TRAIN = cfg['data_path']+'sent_gs_train.json'
PATH_TEST = cfg['data_path']+'sent_gs_test.json'

train_loader = Loader(MAX_LENGTH, INPUT_LEVEL)
test_loader = Loader(MAX_LENGTH, INPUT_LEVEL)
train_loader.readJson(PATH_TRAIN)
test_loader.readJson(PATH_TEST)
    
SRC = Vocab(train_loader.srcs, INPUT_LEVEL, device)
TRG = Vocab(train_loader.trgs, INPUT_LEVEL, device)
SRC.build_vocab()
TRG.build_vocab()

# 하이퍼파라미터 설정
INPUT_DIM = 10000  # 입력 어휘 사이즈
OUTPUT_DIM = 10000  # 출력 어휘 사이즈
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
N_EPOCHS = 12
CLIP = 1

PAD_IDX = TRG.stoi['<pad>']
SOS_IDX = TRG.stoi['<sos>']
EOS_IDX = TRG.stoi['<eos>']

# 장치 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Encoder, Attention, Decoder 인스턴스 생성
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

# Seq2Seq 모델 인스턴스 생성
model = Seq2Seq(enc, dec, SOS_IDX, device, MAX_LENGTH).to(device) 

# 모델 저장
torch.save(model.state_dict(), cfg['data_path']+'s2sAttn_syl_gs_110_128_128')
# # 모델을 저장할 때는 state_dict만 저장합니다.
# torch.save(model.state_dict(), cfg['data_path']+'s2sAttn_syl_gs_110_128_128.pth')

In [ ]:
model.load_state_dict(torch.load(cfg['data_path']+'s2sAttn_syl_gs_110_128_128.pt'))

In [ ]:
import time  # time 모듈을 임포트
from torch.utils.data import DataLoader

def train(model, iterator, optimizer, criterion, clip):
    model.train()    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):

        if i % 10 == 0 or i == len(iterator) - 1:
            print(f'\r{i+1:4}/{len(iterator):4} {(i+1) * 100.0 /len(iterator):.1f}%', end='')

        src = batch[0]
        trg = batch[1]
        
        print('')
        print(src)
        
        #src = src.squeeze(dim=2)  # 3차원으로 변환
        #src = src.transpose(1, 0)  # 2차원으로 변환 (batch 크기 x src 문장 길이)
        
        optimizer.zero_grad()
        output = model(src, trg)        
        #trg = [trg sent len, batch size]
        #output = [trg sent len, batch size, output dim]
        
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)        
        #trg = [(trg sent len - 1) * batch size]
        #output = [(trg sent len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)        
        loss.backward()        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)        
        optimizer.step()
        
        epoch_loss += loss.item()
    print() 
    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    model.eval()    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch[0]
            trg = batch[1]
            
            #src = src.squeeze(dim=2)  # 3차원으로 변환
            #src = src.transpose(1, 0)  # 2차원으로 변환 (batch 크기 x src 문장 길이)

            output = model(src, trg, 0) #turn off teacher forcing
            #trg = [trg sent len, batch size]
            #output = [trg sent len, batch size, output dim]

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)
            #trg = [(trg sent len - 1) * batch size]
            #output = [(trg sent len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def train_model(model,
                train_iterator, valid_iterator,
                optimizer, criterion, CLIP, N_EPOCHS, model_pt_path):
    best_valid_loss = float('inf')
    last_exp_v = float('inf')
    last_exp_t = float('inf')
    inc_streak_v = 0
    inc_streak_t = 0

    for epoch in range(N_EPOCHS):
        start_time = time.time()
        
        train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
        valid_loss = evaluate(model, valid_iterator, criterion)
        
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), model_pt_path)

        exp_t = int(math.exp(train_loss))
        exp_v = int(math.exp(valid_loss))
        torch.save(model.state_dict(), model_pt_path.replace('.pt', f'_E{epoch}T{exp_t}V{exp_v}.pt'))

        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        if exp_v > 20000:
            break 
        if exp_v > last_exp_v:
            inc_streak_v += 1
        else :
            inc_streak_v = 0
        if exp_t > last_exp_t:
            inc_streak_t += 1
        else :
            inc_streak_t = 0
        
        if inc_streak_v > 3 or inc_streak_t > 3:
            break
        last_exp_v = exp_v
        last_exp_t = exp_t
        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

# def collate_batch(batch):
#     src_batch, trg_batch = zip(*batch)
#     src_batch = pad_sequence(src_batch, batch_first=True)
#     trg_batch = pad_sequence(trg_batch, batch_first=True)
#     return src_batch, trg_batch
        
batch_size=32
        
# train_iterator = train_loader.makeIterator(SRC, TRG,
#                                            sos=True, eos=True,
#                                            batch_size=batch_size)
# test_iterator = test_loader.makeIterator(SRC, TRG,
#                                          sos=True, eos=True,
#                                          batch_size=batch_size)
# sos = 1
# eos = 2

train_iterator = train_loader.makeIterator(SRC, TRG, batch_size=batch_size)
test_iterator = test_loader.makeIterator(SRC, TRG, batch_size=batch_size)


# portion = int(len(test_iterator) * 0.5)
# valid_iterator = test_iterator[:portion]
# test_iterator = test_iterator[portion:]

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

N_EPOCHS = 12
CLIP = 1

# model_pt_path = f'./models/{model_name}/{model_name}.pt'
model_pt_path = cfg['data_path'] + 's2sAttn_syl_gs_110_128_128.pt'
        
train_model(model = model, 
            train_iterator = train_iterator, 
            valid_iterator = valid_iterator, 
            optimizer = optimizer, 
            criterion = criterion, 
            CLIP = CLIP, 
            N_EPOCHS = N_EPOCHS, 
            model_pt_path = model_pt_path)

In [ ]:
train_iterator

In [ ]:
# 데이터를 8:2 비율로 분할
train, test = train_test_split(df4, test_size=0.3, random_state=42)

# 분할된 데이터의 크기 확인
print(len(train), len(test))

In [ ]:
# combined_stand.isnull().sum(axis = 0)
# combined_stand.isnull().sum(axis = 0)

In [ ]:
print(len(df_combined))
df_combined = df_combined.dropna(axis=0)
print(len(df_combined))

In [ ]:
df_combined.isnull().sum(axis = 0)

In [ ]:
# 데이터를 8:2 비율로 분할
train, temp = train_test_split(df_combined, test_size=0.3, random_state=42)
valid, test = train_test_split(temp, test_size=0.3, random_state=42)

# 분할된 데이터의 크기 확인
print(train.shape, valid.shape, test.shape)

In [ ]:
# 엑셀 파일로 저장
train.to_excel(cfg['file_path']+'dataset/train_stand2gsd.xlsx', index=False)
valid.to_excel(cfg['file_path']+'dataset/valid_stand2gsd.xlsx', index=False)
test.to_excel(cfg['file_path']+'dataset/test_stand2gsd.xlsx', index=False)

In [ ]:
# 데이터프레임을 TSV 형식으로 저장
train.to_csv(cfg['file_path']+'dataset/train.tsv', sep='\t', index=False)
valid.to_csv(cfg['file_path']+'dataset/test.tsv', sep='\t', index=False)
# test.to_csv(cfg['file_path']+'dataset/test_stand2gsd.tsv', sep='\t', index=False)

In [ ]:
# data = {
#     "stand" : stand_list,
#     "gsd" : gsd_list
# }

In [ ]:
# import json

# # JSON 파일로 저장
# with open(cfg['data_path']+'data.json', 'w', encoding='utf-8') as f:
#     json.dump(data, f, ensure_ascii=False, indent=4)

# print("딕셔너리가 'data.json' 파일로 저장되었습니다.")

In [ ]:
    # # 빈 DataFrame 생성
    # globals()[f'df_{i}'] = pd.DataFrame(columns=['stand', 'gsd'])
    
    # # 문장 분리 및 구두점 유지
    # sentences = re.findall(r'[^.!?]+[.!?]', stand)
    # sentences = [sentence for sentence in sentences if sentence]
    # # 1열에 문장 저장
    # globals()[f'df_{i}']['stand'] = pd.Series(sentences)
    
    # # 문장 분리 및 구두점 유지
    # sentences = re.findall(r'[^.!?]+[.!?]', gsd)
    # sentences = [sentence for sentence in sentences if sentence]
    # # 1열에 문장 저장
    # globals()[f'df_{i}']['gsd'] = pd.Series(sentences)

    # # 연결할 데이터프레임 목록 생성 (제외할 데이터프레임은 건너뜀)
    # dataframes_to_concat = [globals()[f'df_{i}'] for i in range(890, 1737) 
    #                         if i not in (1271, 1273, 1294, 1702)]
    
    # # 데이터프레임 연결
    # df_combined = pd.concat(dataframes_to_concat, ignore_index=True)
    
    # df_combined.shape

In [ ]:
# # 문장 분리 및 구두점 유지
# list_stand = re.findall(r'[^.!?]+[.!?]', stand)
# type(list_stand)

# # 문장 분리 및 구두점 유지
# list_gsd = re.findall(r'[^.!?]+[.!?]', gsd)
# type(list_gsd)

In [ ]:
# data = {
#     "stand" : list_stand,
#     "gsd" : list_gsd,
# }
# data

In [ ]:
# 엑셀 파일로 저장
df_combined.to_excel(cfg['file_path']+'dataset/sentences.xlsx', index=False)

# 토큰화(Tokenization)
링크 : https://data-minggeul.tistory.com/13?category=901822
### (1) 토큰화 (Tokenization)
    - 토큰화란 말뭉치(Corpus)를 토큰(Token)이라는 의미를 가지는 단위로 나누는 작업을 말합니다. 이때 토큰은 단어 혹은 문장 단위가 될 수 있습니다. 하지만 한국어에서는 의미 단위를 구분하기 위해 형태소를 고려해야 합니다.
    - 형태소 단위의 토큰화를 위해서는 KoNLPy 패키지의 Okt, Mecab, Komoran, Hannanum, Kkma와 같은 형태소 분석기를 활용할 수 있습니다. 이 중 Mecab이 속도가 빠른 장점이 있어 주로 활용된다고 합니다.
    - Mecab의 사용법은 아래와 같이 설치후 간단하게 사용할 수 있으며, 다른 형태소 분석기도 유사한 방식으로 사용할 수 있습니다.

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

# https://huggingface.co/skt/kobert-base-v1
tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
morphs = tokenizer.encode(stand)
len(morphs)

### (2) 불용어(Stopword) 처리
    - 문단/문장을 토큰화한 뒤 유의하지 않은 토큰들을 제거하는 작업이 필요합니다. 조사, 접미사와 같이 자주 등장하지만 의미가 없는 토큰들을 불용어라 부르고 학습 전 제거해줍니다.
    - 불용어 처리를 위해 다른 사용자가 정의해놓은 불용어 사전을 활용하거나, 직접 정리한 불용어 사전을 사용하는 방법이 있습니다.

In [ ]:
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize

# 'punkt' 토크나이저 모델 다운로드
nltk.download('punkt')

# 텍스트 파일 열기
with open(cfg['file_path']+'stopword.txt', 'r', encoding='utf-8') as file:
    # 파일의 모든 내용을 읽고 출력하기
    content = file.read()

# 줄바꿈(\n)을 띄어쓰기(' ')로 바꾸기
p = re.compile('\\n')
stopwords = p.sub(' ', content)
stopwords = set(stopwords.split(' '))
tok = word_tokenize(content_8)

def stopword(word_tokenize):
    result = []
    for w in word_tokenize:
        if w not in stopwords:
            result.append(w)
    return result

result = stopword(tok)

print('불용어 제거 전 :',tok) 
print()
print('불용어 제거 후 :',result)